In [ ]:

from keras.datasets import fashion_mnist
from keras.layers import *
from keras.models import Sequential,Model

import numpy as np
import matplotlib.pyplot as plt 

# import matplotlib.pyplot as plt 

import math

print("imported ")
LeakyReLU
print(3)


In [ ]:
from keras.layers import ELU, PReLU, LeakyReLU
from keras.optimizers import Adam
# from keras.layers.advanced_activations import LeakyReLU

In [ ]:
!pip install keras.layers.advanced_activations


In [ ]:

#load dataset
(X_Train, _), (_, _) = fashion_mnist.load_data()

In [ ]:
print(X_Train.shape)

In [ ]:

plt.imshow(X_Train[18], cmap = 'gray')

In [ ]:

X_Train = X_Train.reshape((*X_Train.shape,1)) 
X_Train = (X_Train.astype('float32')-127.5)/127.5

print(np.min(X_Train))
print(np.max(X_Train))

In [ ]:
TOTAL_EPOCHS = 50 
BATCH_SIZE = 256
NO_OF_BATCHES = int(X_Train.shape[0]/BATCH_SIZE)
HALF_BATCH =128

NOISE_DIM = 100
adam = Adam(lr = 2e-4 , beta_1= 0.5)

In [ ]:
#define generator 

generator = Sequential()

generator.add(Dense(7*7*128, input_shape = (NOISE_DIM,)))
generator.add(Reshape((7,7,128)))
generator.add(LeakyReLU(0.2))
generator.add(BatchNormalization())

#double activation size 14 14 64 

generator.add(UpSampling2D())
generator.add(Conv2D(64,kernel_size = (5,5), padding= 'same'))
generator.add(LeakyReLU(0.2))
generator.add(BatchNormalization())

#double activation size 28 28 1 

generator.add(UpSampling2D())
generator.add(Conv2D(1,kernel_size = (5,5), padding= 'same', activation= 'tanh'))



generator.compile(loss = 'binary_crossentropy', optimizer = 'adam')
generator.summary()


# define disc



In [ ]:
#define disc


discriminator = Sequential()
discriminator.add(Conv2D(64,(5,5),strides=(2,2), padding  =  'same', input_shape=(28,28,1)))
discriminator.add(LeakyReLU(0.2))

#next conv layer
discriminator.add(Conv2D(128,(5,5),strides=(2,2), padding  =  'same'))
discriminator.add(LeakyReLU(0.2))

#flatten 

discriminator.add(Flatten())

discriminator.add(Dense(1,activation = 'sigmoid'))

discriminator.compile(loss= 'binary_crossentropy',optimizer = adam)
discriminator.summary()




In [ ]:
discriminator.trainable = False

gan_input = Input(shape = (NOISE_DIM,))
generated_img = generator(gan_input)

gan_output = discriminator(generated_img)
#functional api

model = Model(gan_input,gan_output)
model.compile(loss = 'binary_crossentropy', optimizer = adam)


In [ ]:


def save_imgs(epoch, samples=100):
  noise  = np.random.normal(0,1,size = (samples , NOISE_DIM))
  generated_imgs = generator.predict(noise)
  generated_imgs = generated_imgs.reshape(samples, 28 , 28)
  
  plt.figure(figsize = (10,10))

  for i in range(samples):
    plt.subplot(10,10,i+1)
    plt.imshow(generated_imgs[i], interpolation =  'nearest', cmap = 'gray')
    plt.axis("off")

  plt.tight_layout()
  plt.savefig('images/gan_output_epoch_{0}.png'.format(epoch+1))
  plt.show()
   


In [ ]:
!ls


#Training loop


In [ ]:


d_losses= []
g_losses=[]
for epoch in range(TOTAL_EPOCHS):
    epoch_d_loss = 0
    epoch_g_loss = 0

    #MINI Batch SGD
    for step in range(NO_OF_BATCHES):
      
      # step-1 Disc

      idx = np.random.randint(0, X_Train.shape[0] , HALF_BATCH)
      real_imgs   =  X_Train[idx]

      noise = np.random.normal(0,1,size = (HALF_BATCH, NOISE_DIM))

      fake_imgs = generator.predict(noise)

                                    
      real_y = np.ones((HALF_BATCH,1))*0.9
      fake_y = np.zeros((HALF_BATCH,1))

      #train our disc

      d_loss_real = discriminator.train_on_batch(real_imgs, real_y)
      d_loss_fake = discriminator.train_on_batch(fake_imgs, fake_y)
      d_loss = 0.5*d_loss_real + 0.5*d_loss_fake

      epoch_d_loss +=  d_loss


      #train generator 
      noise  = np.random.normal(0,1,size = (BATCH_SIZE, NOISE_DIM))

      ground_truth_y = np.ones((BATCH_SIZE,1))

      g_loss = model.train_on_batch(noise,ground_truth_y)

      epoch_g_loss += g_loss

    print("epoch %d Disc loss %.4f Generator loss %.4f" %((epoch+1 ), epoch_d_loss/NO_OF_BATCHES, epoch_g_loss/NO_OF_BATCHES))
    d_losses.append(epoch_d_loss/NO_OF_BATCHES)                             
    g_losses.append(epoch_g_loss/NO_OF_BATCHES)                             

    if ( epoch+1) %5 == 0 :
      generator.save('model/gan_generator_{0}.h5'.format(epoch+1))
      save_imgs(epoch)
    






